# Preperation

## ClearML

In [2]:
# https://app.clear.ml/projects/4f45f06bb51f43ecb10dcaf8319c3e76/experiments/a9b2315cdf234266808858d91d90cc74/output/log

from clearml import Task
task = Task.init(project_name='count-pepper', task_name='train')

ClearML Task: created new task id=3ff1d5a9c74c44d7a8016250fb03e9bb
2024-08-07 13:03:35,740 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2024-08-07 13:03:35,766 - clearml.Repository Detection - WARNING - Please install nbconvert using "pip install nbconvert"
2024-08-07 13:03:35,773 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/1aa0d6c5dab5447095488a6887d3fff8/experiments/3ff1d5a9c74c44d7a8016250fb03e9bb/output/log


## Roboflow

In [3]:
import os, yaml
from credentials import credentials
from roboflow import Roboflow

#https://universe.roboflow.com/tecnologico-de-monterrey-f95wv/maturity-peppers-in-greenhouses-by-object-detection/dataset/4

rf = Roboflow(api_key=credentials.get('ROBOFLOW-API-KEY'))
project = rf.workspace("tecnologico-de-monterrey-f95wv").project("maturity-peppers-in-greenhouses-by-object-detection")
version = project.version(4)
dataset = version.download("yolov8")

directory = dataset.location
new_directory_name = directory.rsplit('/', 1)[0] + '/dataset'

os.rename(directory, new_directory_name)

config_file_path = './dataset/data.yaml'

with open(config_file_path, 'r') as f:
    config_file = yaml.safe_load(f)

config_file['path'] = new_directory_name
config_file['test'] = "test/images"
config_file['train'] = "train/images"
config_file['val'] = "valid/images"

with open(config_file_path, "w") as f:
    yaml.dump(config_file, f)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.74, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Maturity-Peppers--in-Greenhouses-by-Object-Detection-4 in yolov8:: 100%|██████████| 9218/9218 [00:01<00:00, 8927.35it/s] 


# Machine Learning

## Load Model

In [1]:
from ultralytics import YOLO

try:
    model = YOLO('./pepper-detection.pt') # load a custom model
except FileNotFoundError:
    model = YOLO('yolov8n.pt') # load a pretrained model


## Train

### From scratch

In [2]:
from ultralytics import YOLO

# Train the model
results = model.train(data="./dataset/data.yaml", epochs=5, batch=-1, device=[0])

Ultralytics YOLOv8.2.74 🚀 Python-3.10.14 torch-2.4.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./dataset/data.yaml, epochs=5, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

/home/takada-labo/projects/count-pepper/.venv/lib/python3.10/site-packages/ultralytics/engine/trainer.py:269: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)


     3011628       8.197         0.191          21.7         35.58        (1, 3, 640, 640)                    list
     3011628       16.39         0.298         21.07          25.8        (2, 3, 640, 640)                    list
     3011628       32.79         0.524         22.06         23.27        (4, 3, 640, 640)                    list
     3011628       65.58         1.036         23.14         27.46        (8, 3, 640, 640)                    list
     3011628       131.2         1.990         30.82          34.8       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 38 for CUDA:0 4.80G/8.00G (60%) ✅


train: Scanning /home/takada-labo/projects/count-pepper/dataset/train/labels.cache... 4026 images, 362 backgrounds, 0 corrupt: 100%|██████████| 4026/4026 [00:00<?, ?it/s]
val: Scanning /home/takada-labo/projects/count-pepper/dataset/valid/labels.cache... 382 images, 24 backgrounds, 0 corrupt: 100%|██████████| 382/382 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00059375), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        382       1312      0.599      0.671      0.639      0.429
             Early-Mid        122        156      0.552      0.583      0.573      0.406
              Immature        201        774      0.728      0.738      0.771      0.505
                Mature        147        272      0.674      0.753      0.739      0.441
              Mid-Late         91        110      0.441      0.609      0.473      0.364
Speed: 0.6ms preprocess, 1.4ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start
2024-08-07 13:10:45,967 - clearml.storage - INFO - Uploading: 0.71MB to runs/detect/train/train_batch1.jpg


████████████████████████████████▊  99% | 0.71/0.71 MB [00:01<00:00,  1.90s/MB]: 


2024-08-07 13:10:52,486 - clearml.storage - INFO - Uploading: 5.96MB to runs/detect/train/weights/best.pt


                                            0% | 0.00/5.96 MB [00:00<?, ?MB/s]: /home/takada-labo/projects/count-pepper/.venv/lib/python3.10/site-packages/tqdm/std.py:636: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
█████████████████████████████████ 100% | 5.96/5.96 MB [00:00<00:00,  6.36MB/s]: 

2024-08-07 13:10:53,442 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/YOLOv8/train.1f20b65d83cc45bfade051ba724e8adb/models/best.pt


### Resume training

In [61]:
import glob
from ultralytics import YOLO

train_folders = glob.glob('./runs/detect/train*')
latest_number = 0
for train_folder in train_folders:
    number = int(0 if train_folder.split('train')[1] == '' else train_folder.split('train')[1])
    latest_number = max(latest_number, number)

last_model_path = f'./runs/detect/train{latest_number}/weights/last.pt' if latest_number != 0 else f'./runs/detect/train/weights/last.pt'

model = YOLO(last_model_path)

results = model.train(resume=True, batch=-1, device=[0])

Ultralytics YOLOv8.2.74 🚀 Python-3.10.14 torch-2.4.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine/trainer: task=detect, mode=train, model=runs/detect/train5/weights/last.pt, data=./dataset/data.yaml, epochs=90, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=runs/detect/train5/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe53c216200>
Traceback (most recent call last):
  File "/home/takada-labo/projects/count-pepper/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "/home/takada-labo/projects/count-pepper/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1441, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.10/selectors.py", line 416, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt: 


## Validate

In [3]:
from ultralytics import YOLO

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
print(metrics.box.map)  # map50-95

Ultralytics YOLOv8.2.52 🚀 Python-3.10.14 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/arcdev/follower-tello-drone/Head_Detection-1/valid/labels.cache... 3529 images, 36 backgrounds, 0 corrupt: 100%|██████████| 3529/3529 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/arcdev/follower-tello-drone/Head_Detection-1/valid/images/1964_jpg.rf.343b2da041c2c2f58a359ddc214c5f15.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /home/arcdev/follower-tello-drone/Head_Detection-1/valid/images/Kamera_17_part_1689243468000_1689244068000_mp4-35_jpg.rf.04389a18dd781138c648df5f181f81cc.jpg: 10 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 221/221 [00:36<00:00,  6.02it/s]


                   all       3529      29429      0.868      0.731      0.807      0.393
Speed: 0.3ms preprocess, 4.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /home/arcdev/follower-tello-drone/runs/detect/val
0.3933452256556003


## Export

In [4]:
import glob
from ultralytics import YOLO

train_folders = glob.glob('./runs/detect/train*')
latest_number = 0
for train_folder in train_folders:
    number = int(0 if train_folder.split('train')[1] == '' else train_folder.split('train')[1])
    latest_number = max(latest_number, number)

best_model_path = f'./runs/detect/train{latest_number}/weights/last.pt' if latest_number != 0 else f'./runs/detect/train/weights/last.pt'

model = YOLO(best_model_path)

model.save("pepper-detection.pt")